In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import ScalarFormatter, MaxNLocator

# Простой прогресс бар без tqdm
def tqdm(iterable, total=None, desc=""):
    """Простая замена tqdm для случаев, когда библиотека недоступна"""
    if total is None:
        total = len(iterable)
    
    for i, item in enumerate(iterable):
        if i % max(1, total // 10) == 0 or i == total - 1:  # Показываем каждые 10%
            print(f"{desc}: {i+1}/{total} ({100*(i+1)/total:.1f}%)")
        yield item

# Функции для вычисления дифракции
def triangle_vertices_cartesian(L):
    """Вершины равностороннего треугольника со стороной L"""
    R = L / np.sqrt(3)
    return np.array([
        [R * np.cos(0), R * np.sin(0)],
        [R * np.cos(2*np.pi/3), R * np.sin(2*np.pi/3)],
        [R * np.cos(4*np.pi/3), R * np.sin(4*np.pi/3)]
    ])

def point_inside_triangle(x, y, vertices):
    """Проверка принадлежности точки треугольнику"""
    v1, v2, v3 = vertices
    cross1 = (v2[0]-v1[0])*(y-v1[1]) - (v2[1]-v1[1])*(x-v1[0])
    cross2 = (v3[0]-v2[0])*(y-v2[1]) - (v3[1]-v2[1])*(x-v2[0])
    cross3 = (v1[0]-v3[0])*(y-v3[1]) - (v1[1]-v3[1])*(x-v3[0])
    return (cross1 >= 0) == (cross2 >= 0) == (cross3 >= 0)

def angular_part(x, y, m):
    """Вычисляет угловую часть e^{imφ} в декартовых координатах"""
    rho = np.sqrt(x**2 + y**2)
    if m >= 0:
        return (x + 1j*y)**m / (rho**m if m != 0 else 1)
    else:
        return (x - 1j*y)**abs(m) / (rho**abs(m))

def compute_bessel_diffraction(vertices, X, Y, z, k, k_z, m_target, N_samples=100):
    """Вычисление дифракции Бесселя с оптимизацией памяти"""
    from scipy.special import jv
    
    x_min, y_min = np.min(vertices, axis=0)
    x_max, y_max = np.max(vertices, axis=0)
    x_ap, y_ap = np.meshgrid(np.linspace(x_min, x_max, N_samples),
                            np.linspace(y_min, y_max, N_samples))
    dx = (x_max - x_min) / (N_samples - 1)
    dy = (y_max - y_min) / (N_samples - 1)

    # Маска для точек внутри треугольника
    mask = np.vectorize(point_inside_triangle, excluded=['vertices'])(x_ap, y_ap, vertices=vertices)

    # Вычисления только для точек внутри
    x_p = x_ap[mask]
    y_p = y_ap[mask]

    # Векторизованные вычисления
    R = np.sqrt(z**2 + (X - x_p[:, None, None])**2 + (Y - y_p[:, None, None])**2)
    R = np.where(R < 1e-10, 1e-10, R)  # Избегаем деления на ноль

    kappa = np.sqrt(k**2 - k_z**2)
    angular_part_val = angular_part(x_p, y_p, m_target)
    bessel_part = jv(abs(m_target), kappa * np.sqrt(x_p**2 + y_p**2))
    exp_part = np.exp(1j * k * R)
    factor = (1j*k - 1/R) * z/R + 1j*k_z

    psi = np.sum(bessel_part[:, None, None] * angular_part_val[:, None, None] * exp_part * factor / (4 * np.pi * R) * dx * dy, axis=0)

    return np.abs(psi)**2

# Параметры расчета
me = 0.511e6           # масса электрона (эВ)
E_k = 100e3            # кинетическая энергия (эВ)
E = 1 + E_k / me       # полная энергия
k = np.sqrt(2*E_k/me)  # волновой вектор
lambda_e = (2*np.pi)/k # длина волны де Бройля для E_k
theta = 0.00001        # угол раствора (рад)
k_z = k * np.cos(theta) # продольная компонента волнового вектора
L = 1e6                # размер апертуры
D = L/np.sqrt(3)       # размер апертуры
z_fraunhofer = (D**2) / (lambda_e)  # граница зон Френеля-Фраунгофера

# Папка и качество вывода
output_dir = "bessel_triangle_zones_small_fonts"
os.makedirs(output_dir, exist_ok=True)
dpi = 300                 # выше dpi
format = "png"             # можно 'png'/'jpg'/'webp'

# Параметры sweep по z (логарифмически для плавности)
z_ref = z_fraunhofer
z_min_factor, z_max_factor = 0.05, 8.0
n_frames = 60            # больше кадров — плавнее
z_factors = np.geomspace(z_min_factor, z_max_factor, n_frames)
z_values = z_factors * z_fraunhofer

# Сетка и интеграция (чуть детальнее)
m_target = 2
N_xy = 200                # больше точек
N_samples = 150

# Масштаб области обзора (одинаковый угловой FOV)
b_ref = 5e6
gamma = 2.0                # компенсация I ∝ 1/z^2

# Геометрия — один раз
vertices = triangle_vertices_cartesian(L)

# 1) Рендерим кадры в память, чтобы зафиксировать общий цветовой масштаб
print("Вычисление кадров...")
frames = []
for idx, z in tqdm(enumerate(z_values), total=len(z_values), desc="Вычисление"):
    b = b_ref * (z / z_ref)
    x = np.linspace(-b, b, N_xy)
    y = np.linspace(-b, b, N_xy)
    X, Y = np.meshgrid(x, y)

    out = compute_bessel_diffraction(vertices, X, Y, z, k, k_z, m_target, N_samples)
    I = out if not np.iscomplexobj(out) else np.abs(out)**2
    I = I * (z / z_ref)**gamma
    frames.append((I, b, z))

# Общий робастный максимум — без мерцания цветов
vmax = np.percentile(np.concatenate([I.ravel() for I, _, _ in frames]), 99.5)

# 2) Сохраняем кадры с уменьшенными шрифтами
print("Сохранение кадров...")
single_figsize = (8, 6)
for idx, (I, b, z) in tqdm(enumerate(frames), total=len(frames), desc="Сохранение"):
    fig, ax = plt.subplots(figsize=single_figsize)
    im = ax.imshow(I, extent=[-b, b, -b, b], cmap="viridis", origin="lower", vmin=0, vmax=vmax)

    # УМЕНЬШЕННЫЕ РАЗМЕРЫ ШРИФТОВ
    ax.set_title(f"$\\ell$={m_target}: z = {z*386e-15:.2e} m", fontsize=16)  # было 25
    ax.set_xlabel("X position (pm)", fontsize=14)  # было 25
    ax.set_ylabel("Y position (pm)", fontsize=14)  # было 25
    ax.tick_params(axis="both", which="major", labelsize=12)  # было 20
    ax.xaxis.set_major_locator(MaxNLocator(5))
    ax.yaxis.set_major_locator(MaxNLocator(5))
    ax.xaxis.get_offset_text().set_fontsize(12)  # было 20
    ax.yaxis.get_offset_text().set_fontsize(12)  # было 20

    cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label("Intensity", fontsize=14)  # было 25
    cbar.ax.tick_params(labelsize=12)  # было 20
    cbar.formatter = ScalarFormatter(useMathText=True)
    cbar.formatter.set_powerlimits((-3, 3))
    cbar.formatter.set_scientific(True)
    cbar.formatter.set_useOffset(False)
    cbar.update_normal(im)

    plt.tight_layout()
    # Имя с нулевым доп. индексом — удобно для ffmpeg
    filename = f"frame_{idx:04d}.png"
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath, dpi=dpi, bbox_inches="tight", pad_inches=0.1)
    plt.close()

print(f"Сохранено {len(frames)} кадров в '{output_dir}'")
print("Все шрифты уменьшены:")
print("- Заголовок: 25 → 16")
print("- Подписи осей: 25 → 14") 
print("- Цифры на осях: 20 → 12")
print("- Подписи цветовой шкалы: 25 → 14")
print("- Цифры на цветовой шкале: 20 → 12")

Вычисление кадров...
Вычисление: 1/60 (1.7%)
Вычисление: 7/60 (11.7%)
Вычисление: 13/60 (21.7%)
Вычисление: 19/60 (31.7%)
Вычисление: 25/60 (41.7%)
Вычисление: 31/60 (51.7%)
Вычисление: 37/60 (61.7%)
Вычисление: 43/60 (71.7%)
Вычисление: 49/60 (81.7%)
Вычисление: 55/60 (91.7%)
Вычисление: 60/60 (100.0%)
Сохранение кадров...
Сохранение: 1/60 (1.7%)
Сохранение: 7/60 (11.7%)
Сохранение: 13/60 (21.7%)
Сохранение: 19/60 (31.7%)
Сохранение: 25/60 (41.7%)
Сохранение: 31/60 (51.7%)
Сохранение: 37/60 (61.7%)
Сохранение: 43/60 (71.7%)
Сохранение: 49/60 (81.7%)
Сохранение: 55/60 (91.7%)
Сохранение: 60/60 (100.0%)
Сохранено 60 кадров в 'bessel_triangle_zones_small_fonts'
Все шрифты уменьшены:
- Заголовок: 25 → 16
- Подписи осей: 25 → 14
- Цифры на осях: 20 → 12
- Подписи цветовой шкалы: 25 → 14
- Цифры на цветовой шкале: 20 → 12
